In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
data = [9, 58, 75, 110, 112, 132, 151, 276, 281]
av = [200, 200, 200, 200, 200, 200, 200, 200, 200, 200]
data_a = [i-200 for i in data]
data_a

[-191, -142, -125, -90, -88, -68, -49, 76, 81]

In [3]:
stats.wilcoxon(data_a)

/Users/alina/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


WilcoxonResult(statistic=7.0, pvalue=0.0663160312989308)

In [52]:
np.median(data_a)

-88.0

In [8]:
d1 = [22,22,15,13,19,19,18,20,21,13,13,15]
d2 = [17,18,18,15,12,4,14,15,10]

In [9]:
stats.mannwhitneyu(d1, d2, alternative='greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.02900499272087373)

In [10]:
data = pd.read_csv('challenger.txt', sep = '\t', header = 0)

In [11]:
data

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0
5,Jun18.83,22.2,0
6,Aug30.83,22.8,0
7,Nov28.83,21.1,0
8,Feb03.84,13.9,1
9,Apr06.84,17.2,1


In [12]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [13]:
np.random.seed(0)
data_1 = list(map(np.mean, get_bootstrap_samples(data[data.Incident == 1].Temperature.values, 1000)))
data_2 = list(map(np.mean, get_bootstrap_samples(data[data.Incident == 0].Temperature.values, 1000)))
delta_mean_scores = list(map(lambda x: x[1] - x[0], zip(data_1, data_2)))

print("95% confidence interval for the difference between means",  stat_intervals(delta_mean_scores, 0.05))

95% confidence interval for the difference between means [1.45040179 8.06457589]


In [14]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), list(filter(lambda i: i not in index, range(n)))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [15]:
np.random.seed(0)
print("p-value: %f" % permutation_test(data_1, data_2, max_permutations = 10000))

p-value: 0.000100
